# Before runing the notebook make sure u have aws configuration


In [ ]:
import boto3 

# UTILS
---
#### Create dynamodb database

In [ ]:

KEYS = [('node_id','HASH','S'), ('node_dns','RANGE','S')] # (NAME, KEY_TYPE, ATTR_TYPE)
TABLE_NAME = 'nodes'
RW_CAPACITY = 5

def create_nodes_db():
    dynamodb = boto3.resource('dynamodb')
    try:
        table = dynamodb.create_table(
                    TableName=TABLE_NAME,
                    KeySchema=[{'AttributeName':k[0],
                                'KeyType':k[1]} for k in KEYS],

                    AttributeDefinitions=[{'AttributeName':k[0],
                                           'AttributeType':k[2]} for k in KEYS],

                    ProvisionedThroughput={
                        'ReadCapacityUnits': RW_CAPACITY,
                        'WriteCapacityUnits': RW_CAPACITY})


        table.meta.client.get_waiter('table_exists').wait(TableName=TABLE_NAME)
        return table
    except dynamodb_client.exceptions.ResourceInUseException:
#         print(f'{TABLE_NAME} DB Already exists')
        return dynamodb.Table(TABLE_NAME)
     
table = create_nodes_db()

# Init Instance test
assumptions:
 - DB: Table 'node' were created


In [ ]:
import time

myMessages = None
# Dummy funcs
def get_instance_ID():
    return 2
def get_public_DNS():
    return '127.0.0.2'

def create_sqs():
    private_sqs_params = dict(
      QueueName=f'{get_instance_ID()}.fifo',
      Attributes={
          'FifoQueue': 'True'
      },
    )
    
    # Get the service resource
    sqs = boto3.resource('sqs')
    return sqs.create_queue(**private_sqs_params)
    
def init_instance(table = 'nodes'):
    myMessages = create_sqs()
    dynamodb = boto3.resource('dynamodb')
    while True:
        try:
            
            table = dynamodb.Table(table)
            break
        except:
            time.sleep(5)
            continue
            
    
    response = table.put_item(
           Item={
                'node_id': str(get_instance_ID()),
                'node_dns': get_public_DNS(),
                'message_q': f'{get_instance_ID()}.fifo',
                'role': 'user',
            }
        )
    
    print('Node initialized')


In [ ]:
messages = create_sqs()

In [ ]:
init_instance()